In [1]:
from pymongo import MongoClient
import pymongo
from cartoframes.viz import Map,Layer, popup_element

import pandas as pd
import geopandas as gpd
from pymongo import GEOSPHERE
import shapely.geometry
import warnings
warnings.filterwarnings("ignore")

import requests
import json
from dotenv import load_dotenv
import os


In [2]:
import funcionesrequest as fr

### De la base de datos de empresas, separamos todas las oficinas y creamos una nueva coleccion con ellas.

In [3]:
client = MongoClient()

companies = client.ironhack.companies
office = client.ironhack.office
pruebaoffice =  client.ironhack.pruebaoffice

In [4]:
res = companies.aggregate([
    {"$unwind":"$offices"},
    {"$match":{"offices.latitude":{"$ne":None}, "offices.longitude":{"$ne":None}}},
    {"$project":{"_id":0}}])

In [5]:
pruebaoffice.insert_many(res)

In [6]:
res = pruebaoffice.find({},{"offices":1})

In [7]:
res = list(res)

In [8]:
res[:2]

[{'_id': ObjectId('60218e79998f0fe3c64fde69'),
  'offices': {'description': '',
   'address1': '710 - 2nd Avenue',
   'address2': 'Suite 1100',
   'zip_code': '98104',
   'city': 'Seattle',
   'state_code': 'WA',
   'country_code': 'USA',
   'latitude': 47.603122,
   'longitude': -122.333253}},
 {'_id': ObjectId('60218e79998f0fe3c64fde6a'),
  'offices': {'description': '',
   'address1': '270 Lafayette Street',
   'address2': 'Suite 505',
   'zip_code': '10012',
   'city': 'New York',
   'state_code': 'NY',
   'country_code': 'USA',
   'latitude': 40.7237306,
   'longitude': -73.9964312}}]

### Añadimos las coordenada con en el tipo `point`

In [9]:
for comp in res: 
    geojson = {
        "type":"Point",
        "coordinates":[comp["offices"]["longitude"], comp["offices"]["latitude"]]
    }
    pruebaoffice.update_one(comp, {"$set":{"geojson":geojson}})

In [10]:
pruebaoffice.create_index([("geojson", "2dsphere")])

'geojson_2dsphere'

### He decidido que la ubicacion de la empresa estrá en Londres.  
#### Como no tengo unas coordenadas de referencia voy a realizar la busqueda de tos los `Starbucks` de Londres usando el paramentro `near` a la hora de hacer la llamada a la API de FourSquare



In [11]:
load_dotenv()

True

In [12]:
tok1 = os.getenv("tok1")
tok2 = os.getenv("tok2")

In [13]:
enl = 'https://api.foursquare.com/v2/venues/explore'

Creamos los parametros para la busqueda.

In [14]:
st_parametros ={"client_id" : tok1,
              "client_secret": tok2,
              "v": "20180323",
              "near": "london",
              "query": "starbucks" ,
              "radius": 300,
              "limit": 50
             
             }

In [15]:
from functools import reduce
import operator

In [16]:

result_st = requests.get(url = enl, params = st_parametros ).json()["response"]["groups"][0]["items"]

name = ['venue','name']
lat = ['venue','location','lat']
long = ['venue','location','lng']
#cat = ['venue', 'categories', 'name']


In [17]:
def getlistdata(result, category_name):
    
    listdata = []

    for dic in result:
        dictdata = {}

        dictdata["name"]= fr.getFromDict(dic, name)
        dictdata["latitud"]= fr.getFromDict(dic, lat)
        dictdata["longitud"]= fr.getFromDict(dic, long)
        dictdata["category"]= category_name
    
        listdata.append(dictdata)
        
    return listdata

In [18]:
starb = getlistdata(result_st, "starbucks")

In [19]:
starb[:5]

[{'name': 'Starbucks',
  'latitud': 51.50745,
  'longitud': -0.122863,
  'category': 'starbucks'},
 {'name': 'Starbucks',
  'latitud': 51.512332,
  'longitud': -0.121685,
  'category': 'starbucks'},
 {'name': 'Starbucks',
  'latitud': 51.513868,
  'longitud': -0.117669,
  'category': 'starbucks'},
 {'name': 'Starbucks',
  'latitud': 51.515233,
  'longitud': -0.130191,
  'category': 'starbucks'},
 {'name': 'Starbucks',
  'latitud': 51.510288,
  'longitud': -0.139154,
  'category': 'starbucks'}]

In [20]:
df_st = pd.DataFrame(starb)

df_st.head()

,name,latitud,longitud,category
0,Starbucks,51.507450,-0.122863,starbucks
1,Starbucks,51.512332,-0.121685,starbucks
2,Starbucks,51.513868,-0.117669,starbucks
3,Starbucks,51.515233,-0.130191,starbucks
4,Starbucks,51.510288,-0.139154,starbucks


In [21]:
gdf_st = gpd.GeoDataFrame(df_st, geometry = gpd.points_from_xy(df_st.longitud, df_st.latitud))

## A partir de ahora este paso lo haremos con una función.

In [22]:
gdf_st.head()

,name,latitud,longitud,category,geometry
0,Starbucks,51.507450,-0.122863,starbucks,POINT (-0.12286 51.50745)
1,Starbucks,51.512332,-0.121685,starbucks,POINT (-0.12169 51.51233)
2,Starbucks,51.513868,-0.117669,starbucks,POINT (-0.11767 51.51387)
3,Starbucks,51.515233,-0.130191,starbucks,POINT (-0.13019 51.51523)
4,Starbucks,51.510288,-0.139154,starbucks,POINT (-0.13915 51.51029)


In [23]:
Map(Layer(gdf_st, popup_hover = [popup_element ("longitud")]))

In [24]:
gdf_st['geometry']=gdf_st['geometry'].apply(lambda x:shapely.geometry.mapping(x))


In [25]:
## 51.513 , -0.133

## Realizo busquedas para encontrar bares y colegios cerca de las coordenada `51.513 , -0.133` que es un Starbucks en el barrio de Soho, Londres. 

In [26]:
myloc = {'type': 'Point', 'coordinates': [-0.133, 51.513]}

## Buscamos ocio nocturo cerca de la localización. 

In [27]:
party_parametros ={"client_id" : tok1,
              "client_secret": tok2,
              "v": "20180323",
              "ll": f"{myloc.get('coordinates')[1]},{myloc.get('coordinates')[0]}",
              
              "query": "Nightlife Spot",
              "radius": 500,
              "limit": 50
             
             }

In [28]:
result_party = requests.get(url = enl, params = party_parametros ).json()["response"]["groups"][0]["items"]

name = ['venue','name']
lat = ['venue','location','lat']
long = ['venue','location','lng']



In [29]:
party = getlistdata(result_party, "nightlife_spot")

In [30]:
df_party = pd.DataFrame(party)

df_party.head()

,name,latitud,longitud,category
0,Opium Cocktails & Dim Sum Parlour,51.511742,-0.131384,nightlife_spot
1,Peony,51.511780,-0.131511,nightlife_spot
2,The Vault,51.514780,-0.131367,nightlife_spot
3,The Blind Pig,51.513788,-0.136620,nightlife_spot
4,Duke of Wellington,51.512320,-0.133117,nightlife_spot


In [31]:
gdf_partymap = fr.makegeomap(df_party)
gdf_partymap.head()

,name,latitud,longitud,category,geometry
0,Opium Cocktails & Dim Sum Parlour,51.511742,-0.131384,nightlife_spot,POINT (-0.13138 51.51174)
1,Peony,51.511780,-0.131511,nightlife_spot,POINT (-0.13151 51.51178)
2,The Vault,51.514780,-0.131367,nightlife_spot,POINT (-0.13137 51.51478)
3,The Blind Pig,51.513788,-0.136620,nightlife_spot,POINT (-0.13662 51.51379)
4,Duke of Wellington,51.512320,-0.133117,nightlife_spot,POINT (-0.13312 51.51232)


In [32]:
party_map = Map(Layer(gdf_partymap, popup_hover = [popup_element ("name", "night club")]))
party_map

In [33]:
gdf_party = gdf_partymap
gdf_party['geometry']=gdf_partymap['geometry'].apply(lambda x:shapely.geometry.mapping(x))


## Buscamos colegios cerca

In [34]:
school_parametros ={"client_id" : tok1,
              "client_secret": tok2,
              "v": "20180323",
              "ll": f"{myloc.get('coordinates')[1]},{myloc.get('coordinates')[0]}",
              
              "query": "Elementary school",
              "radius": 1000,
              "limit": 50
             
             }

In [35]:
result_school = requests.get(url = enl, params = school_parametros ).json()["response"]["groups"][0]["items"]

name = ['venue','name']
lat = ['venue','location','lat']
long = ['venue','location','lng']




In [36]:
school= getlistdata(result_school, "elementary_school")

In [37]:
df_school = pd.DataFrame(school)

df_school.head()

,name,latitud,longitud,category
0,Soho Parish Primary School,51.511500,-0.134110,elementary_school
1,nwbsm emergency electricians,51.517382,-0.130463,elementary_school
2,ZIK Expert Teachers Ltd,51.514890,-0.123537,elementary_school
3,"UK College for Health, Medical & Research",51.518471,-0.142298,elementary_school


In [38]:
gdf_schoolmap = fr.makegeomap(df_school)

In [39]:
school_map = Map(Layer(gdf_schoolmap, popup_hover = [popup_element ("name", "school")]))
school_map

In [40]:
gdf_school = gdf_schoolmap
gdf_school['geometry']=gdf_schoolmap['geometry'].apply(lambda x:shapely.geometry.mapping(x))


## Buscamos restaurantes veganos

In [41]:
vegan_parametros ={"client_id" : tok1,
              "client_secret": tok2,
              "v": "20180323",
              "ll": f"{myloc.get('coordinates')[1]},{myloc.get('coordinates')[0]}",
              
              "query": "Vegan Restaurant",
              "radius": 1000,
              "limit": 50
             
             }

In [42]:
result_vegan = requests.get(url = enl, params = vegan_parametros ).json()["response"]["groups"][0]["items"]

name = ['venue','name']
lat = ['venue','location','lat']
long = ['venue','location','lng']



In [43]:
vegan= getlistdata(result_vegan, "vegan")

In [44]:
df_vegan = pd.DataFrame(vegan)

df_vegan.head()

,name,latitud,longitud,category
0,Mildreds,51.513034,-0.136392,vegan
1,Ugly Dumpling,51.513352,-0.138407,vegan
2,Yorica,51.514273,-0.134633,vegan
3,Maoz,51.512861,-0.132200,vegan
4,Ethos,51.516410,-0.139172,vegan


In [45]:
gdf_veganmap = fr.makegeomap(df_vegan)

In [46]:
vegan_map = Map(Layer(gdf_veganmap, popup_hover = [popup_element ("name", "Vegan")]))
vegan_map

In [47]:
gdf_vegan = gdf_veganmap
gdf_vegan['geometry']=gdf_vegan['geometry'].apply(lambda x:shapely.geometry.mapping(x))


## Creo DB `geo`y añado 1 colección llamada `places` con: `party`  , `school`  ,  `vegan` y `Starbucks`


In [48]:
client=MongoClient()
db = client.geo
collection = db.places
collection.create_index([("geometry", GEOSPHERE)])



'geometry_2dsphere'

In [49]:
data = gdf_party.to_dict(orient='records')  #Para añadir party a MongoDB
collection.insert_many(data)

data2 = gdf_school.to_dict(orient='records')  #Para añadir school a MongoDB
collection.insert_many(data2)

data3 = gdf_vegan.to_dict(orient='records')  #Para añadir vegan a MongoDB
collection.insert_many(data3)

data5 = gdf_st.to_dict(orient='records')  #Para añadir starbucks a MongoDB
collection.insert_many(data5)

## Encontrar oficinas en un radio de 200m de las coordenadas elegidas  `51.513 , -0.133` , para obtener nuestros candidatos.

In [50]:
location = {
        "type":"Point",
        "coordinates":[ -0.133 , 51.513 ]
    }

En la base de datos donde subimos las oficinas de cada empresa hacemos una colsulta.

In [51]:
res = pruebaoffice.find({"geojson":{"$near":location,"$maxDistance":200}}, {"name":1,"offices":1})

In [52]:
res = list(res)
#res

In [53]:
company_name = ['name']
company_address = ['offices', 'address1']
company_lat = ['offices', 'latitude']
company_long = ['offices', 'longitude']

In [54]:
companies_ = []

for dic in res:
    e = {}
    
    e["name"]= fr.getFromDict(dic, company_name )
    e["latitud"]= fr.getFromDict(dic, company_lat)
    e["longitud"]= fr.getFromDict(dic, company_long)
    e["address"]= fr.getFromDict(dic, company_address)
    
    
    companies_.append(e)

companies_[:5]

[{'name': '111pix',
  'latitud': 51.5133527,
  'longitud': -0.1339263,
  'address': '111 Wardour Street'},
 {'name': 'Team Rubber',
  'latitud': 51.5137373,
  'longitud': -0.1316318,
  'address': '50 Frith Street'},
 {'name': 'Undertone',
  'latitud': 51.5118239,
  'longitud': -0.132904,
  'address': '39-45 Shaftesbury Avenue'},
 {'name': 'muzu tv',
  'latitud': 51.513678,
  'longitud': -0.13063,
  'address': '22 Greek Street'}]

### Después de tratar los datos, obtenemos 4 candidatos:

In [55]:
df_companies = pd.DataFrame(companies_)

df_companies.head()

,name,latitud,longitud,address
0,111pix,51.513353,-0.133926,111 Wardour Street
1,Team Rubber,51.513737,-0.131632,50 Frith Street
2,Undertone,51.511824,-0.132904,39-45 Shaftesbury Avenue
3,muzu tv,51.513678,-0.130630,22 Greek Street


In [56]:
gdf_companiesmap = fr.makegeomap(df_companies)

In [57]:
companies_map = Map(Layer(gdf_companiesmap, popup_hover = [popup_element ("name", "Company")]))
companies_map 

In [58]:
gdf_companies = gdf_companiesmap
gdf_companies['geometry']=gdf_companies['geometry'].apply(lambda x:shapely.geometry.mapping(x))


#### Creo una colección nueva dentro de la DB `geo`, solo con las 4 oficinas candidatas.

In [59]:
collection_offices = db.offices_final
collection_offices.create_index([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [60]:
data4 = gdf_companies.to_dict(orient='records')
collection_offices.insert_many(data4)

In [61]:
import geopy.distance

### ¿Qué hay cerca de cada oficina? 

Con `geopy.distance` podemos calcular la distancia en metros entre dos coordenadas.

In [62]:
gdf_st.head()

,name,latitud,longitud,category,geometry
0,Starbucks,51.507450,-0.122863,starbucks,"{'type': 'Point', 'coordinates': (-0.122863, 5..."
1,Starbucks,51.512332,-0.121685,starbucks,"{'type': 'Point', 'coordinates': (-0.121685, 5..."
2,Starbucks,51.513868,-0.117669,starbucks,"{'type': 'Point', 'coordinates': (-0.117669, 5..."
3,Starbucks,51.515233,-0.130191,starbucks,"{'type': 'Point', 'coordinates': (-0.130191, 5..."
4,Starbucks,51.510288,-0.139154,starbucks,"{'type': 'Point', 'coordinates': (-0.139154, 5..."


In [63]:
gdf_companies

,name,latitud,longitud,address,geometry
0,111pix,51.513353,-0.133926,111 Wardour Street,"{'type': 'Point', 'coordinates': (-0.1339263, ..."
1,Team Rubber,51.513737,-0.131632,50 Frith Street,"{'type': 'Point', 'coordinates': (-0.1316318, ..."
2,Undertone,51.511824,-0.132904,39-45 Shaftesbury Avenue,"{'type': 'Point', 'coordinates': (-0.132904, 5..."
3,muzu tv,51.513678,-0.130630,22 Greek Street,"{'type': 'Point', 'coordinates': (-0.13063, 51..."


### ¿Cuantos Starbucks hay a menos de 150m de cada oficina?.

El numero de veces que aparezca el nombre de una empresa en la lista `score` corresponde al nº de Starbucks que se encuentran cerca. 

In [64]:
score = []

for comp in gdf_companies.index:
    cord = []
    cord.append(gdf_companies.longitud[comp])
    cord.append(gdf_companies.latitud[comp])
    
    for st in gdf_st.index:
        cord_st = []
        cord_st.append(gdf_st.longitud[st])
        cord_st.append(gdf_st.latitud[st])

        dist = geopy.distance.geodesic(cord, cord_st)

        if dist <= 0.150:
            score.append(gdf_companies.name[comp])

print(score) 

['111pix', 'Undertone']


In [65]:
def contar(lista):
    cuenta = {
        "111pix": 0,
        "Team Rubber":0,
        "Undertone" :0,
        "muzu tv": 0
    }

    for name in lista: 
        if name == "111pix":
            cuenta[name] += 1
        elif name == "Team Rubber":
            cuenta[name] += 1
        elif name == "Undertone":
            cuenta[name] += 1
        else :
            cuenta[name] +=1
    
    return cuenta

In [66]:
starbucks = contar(score)
starbucks

{'111pix': 1, 'Team Rubber': 0, 'Undertone': 1, 'muzu tv': 0}

### ¿Cuantos bares de copas hay a menos de 500m. de cada oficina?

In [67]:
score = []

for comp in gdf_companies.index:
    cord = []
    cord.append(gdf_companies.longitud[comp])
    cord.append(gdf_companies.latitud[comp])
    
    for party in gdf_party.index:
        cord_party = []
        cord_party.append(gdf_party.longitud[party])
        cord_party.append(gdf_party.latitud[party])
        
        dist = geopy.distance.geodesic(cord, cord_party)
        
        if dist <= 0.500:
            score.append(gdf_companies.name[comp])
#print(score) 

In [68]:
party = contar(score)
party

{'111pix': 40, 'Team Rubber': 40, 'Undertone': 39, 'muzu tv': 41}

### ¿Cuantos colegios hay a menos de 500 metros de cada oficina?

In [69]:
score = []

for comp in gdf_companies.index:
    cord = []
    cord.append(gdf_companies.longitud[comp])
    cord.append(gdf_companies.latitud[comp])
    
    for school in gdf_school.index:
        cord_school = []
        cord_school.append(gdf_school.longitud[school])
        cord_school.append(gdf_school.latitud[school])
        
        dist = geopy.distance.geodesic(cord, cord_school)
        
        if dist <= 0.5:
            score.append(gdf_companies.name[comp])
print(score)

['111pix', 'Team Rubber', 'Team Rubber', 'Undertone', 'muzu tv', 'muzu tv']


In [70]:
school = contar(score)
school

{'111pix': 1, 'Team Rubber': 2, 'Undertone': 1, 'muzu tv': 2}

### ¿ Cuantos restaurantes veganos hay a menos de 500 metros de cada oficina?

In [71]:
score = []

for comp in gdf_companies.index:
    cord = []
    cord.append(gdf_companies.longitud[comp])
    cord.append(gdf_companies.latitud[comp])
    
    for vegan in gdf_vegan.index:
        cord_vegan = []
        cord_vegan.append(gdf_vegan.longitud[vegan])
        cord_vegan.append(gdf_vegan.latitud[vegan])
        
        dist = geopy.distance.geodesic(cord, cord_vegan)
        
        if dist <= 0.500:
            score.append(gdf_companies.name[comp])
#print(score)

In [72]:
vegan = contar(score)
vegan

{'111pix': 11, 'Team Rubber': 7, 'Undertone': 9, 'muzu tv': 9}

In [73]:
print(starbucks)
print(party)
print(school)
print(vegan)

{'111pix': 1, 'Team Rubber': 0, 'Undertone': 1, 'muzu tv': 0}
{'111pix': 40, 'Team Rubber': 40, 'Undertone': 39, 'muzu tv': 41}
{'111pix': 1, 'Team Rubber': 2, 'Undertone': 1, 'muzu tv': 2}
{'111pix': 11, 'Team Rubber': 7, 'Undertone': 9, 'muzu tv': 9}


In [74]:
datafinal= pd.DataFrame([starbucks, party,school,vegan], index = ["starbucks", "party", " school", "vegan"])

In [75]:
datafinal

,111pix,Team Rubber,Undertone,muzu tv
starbucks,1,0,1,0
party,40,40,39,41
school,1,2,1,2
vegan,11,7,9,9


In [76]:
datafinal = datafinal.T

### Quitamos los que no tienen sturbucks a menos de 150m

In [78]:
datafinal = datafinal[(datafinal["starbucks"]>=1)]

In [79]:
datafinal.columns

Index(['starbucks', 'party', ' school', 'vegan'], dtype='object')

### Ponemos puntuacion por importancia.   Damos 100 puntos por cada colegio , 40 puntos por cada bar y 30 puntos por cada vegano

In [81]:
datafinal["party"]= datafinal["party"].apply(lambda x :x* 40)
datafinal[" school"]= datafinal[" school"].apply(lambda x :x* 100)
datafinal["vegan"]= datafinal["vegan"].apply(lambda x :x* 30)

datafinal= datafinal.T

In [82]:
datafinal

,111pix,Undertone
starbucks,1,1
party,1600,1560
school,100,100
vegan,330,270


In [83]:
datafinal = datafinal.sum()

In [84]:
datafinal

111pix       2031
Undertone    1931
dtype: int64

#### Buscamos las coordenadas en la colección de MongoDB creada anteriormente

In [85]:
result = list(collection_offices.find({"name": "111pix"})) 

In [86]:
result

[{'_id': ObjectId('60218e98998f0fe3c650093c'),
  'name': '111pix',
  'latitud': 51.5133527,
  'longitud': -0.1339263,
  'address': '111 Wardour Street',
  'geometry': {'type': 'Point', 'coordinates': [-0.1339263, 51.5133527]}}]

## Usamos `Folium` para crear el mapa final que muestre todo.


In [87]:
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

In [88]:
#coordinates:[ -0.133 , 51.513 ]

inicial_lat = 51.51339263 
inicial_long = -0.1339263

In [89]:
loc = [inicial_lat, inicial_long]

### Creación de mapa vacío, con las coordenadas de la oficina elegida. 

In [90]:
map_1 = Map(location = [inicial_lat, inicial_long], zoom_start = 17 )

#map_1

Concatenamos los dataframes de Starbucks, pubs, veganos y colegios para despues poder iterar en la columna category y crear una etiqueta diferente para cada tipo.

In [91]:
df_final = pd.concat([gdf_st, gdf_school, gdf_party, gdf_vegan], axis=0)

In [92]:
df_final

,name,latitud,longitud,category,geometry
0,Starbucks,51.507450,-0.122863,starbucks,"{'type': 'Point', 'coordinates': (-0.122863, 5..."
1,Starbucks,51.512332,-0.121685,starbucks,"{'type': 'Point', 'coordinates': (-0.121685, 5..."
2,Starbucks,51.513868,-0.117669,starbucks,"{'type': 'Point', 'coordinates': (-0.117669, 5..."
3,Starbucks,51.515233,-0.130191,starbucks,"{'type': 'Point', 'coordinates': (-0.130191, 5..."
4,Starbucks,51.510288,-0.139154,starbucks,"{'type': 'Point', 'coordinates': (-0.139154, 5..."
...,...,...,...,...,...
19,Veggie Pret,51.513270,-0.136228,vegan,"{'type': 'Point', 'coordinates': (-0.136228465..."
20,by CHLOE. Covent Garden,51.513552,-0.120383,vegan,"{'type': 'Point', 'coordinates': (-0.120382905..."
21,Kalifornia Kitchen,51.518578,-0.132557,vegan,"{'type': 'Point', 'coordinates': (-0.13255745,..."
22,Misato,51.510896,-0.131654,vegan,"{'type': 'Point', 'coordinates': (-0.131653895..."


In [93]:
for i,row in df_final.iterrows():
    
    place = {
        "location": [row["latitud"], row["longitud"]],
        "tooltip": row["category"]
    }
    
    
    if row["category"] == "starbucks":
        icon = Icon(color = "green",
                    prefix = "fa",
                    icon = "coffee",
                    icon_color = "black"
                   )
    
    elif row["category"] == "elementary_school":
        icon = Icon(color = "blue",
                    prefix = "fa",
                    icon = "child",
                    icon_color = "white"
                   )
    
    elif row["category"] == "nightlife_spot":
        icon = Icon(color = "beigie",
                    prefix = "fa",
                    icon = "glass",
                    icon_color = "black"
                   )
        
    else: 
        icon = Icon(color = "orange",
                    prefix = "fa",
                    icon = "leaf",
                    icon_color = "black"
                   )
    Marker(**place, icon = icon).add_to(map_1)
#map_1        

### Ahora añadimos la posición de la oficina

In [94]:
icono = Icon( color="black",
              prefix = "fa",
              icon = "suitcase",
              icon_color = "white",
              tooltip = "OFICINA"
            )

In [95]:
oficina = Marker(location=loc, icon = icono, tooltip= "Oficina ")

oficina.add_to(map_1)  

In [96]:
map_1

In [101]:
map_1.save('maps/mapa_final.html')